This notebook implements national land cover prediciton using the pre-trained random forest model generated from previous step. DE Africa's semiannual MADs products are included as features.

### load packages

In [ ]:
%matplotlib inline
import os
import datacube
import warnings
import time
import numpy as np
from scipy import stats
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from odc.io.cgroups import get_cpu_quota
from odc.algo import xr_geomedian
import xarray as xr
from joblib import load
from deafrica_tools.classification import predict_xr
from deafrica_tools.dask import create_local_dask_cluster
from deafrica_tools.datahandling import load_ard
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.plotting import display_map
from datacube.utils.cog import write_cog

### load data and set parameters

In [ ]:
# file paths and attributes
lesotho_tiles_shp='Data/Mozambique_50km_sample_regions.geojson' # covering the entire country
rf_model_path= 'Results/RF_model_using_filtered_15ptc_td_Mozambique_2021.joblib' # trained random forest model

# class_name = 'LC_Class_I' # class label in integer format
class_name = 'Class_I' # class label in integer format
crs='epsg:32736' # # output crs: WGS84/UTM Zone 36S
# fill_nan_value=-999 # value to replace nans in query results

# load and get bounding boxes of tiles covering Mozambique
lesotho_tiles=gpd.read_file(lesotho_tiles_shp).to_crs('epsg:4326')
tile_bboxes=lesotho_tiles.bounds
print('tile boundaries for Mozambique: \n',tile_bboxes)

# load trained classifier
rf_models = load(rf_model_path).set_params(n_jobs=1)
print('loaded random forest model:\n',rf_models)

### define feature layer function - same as extracting features

In [ ]:
# define a function to feature layers
def feature_layers(query):
    measurements = ['blue','green','red','red_edge_1','red_edge_2', 'red_edge_3','nir_1','swir_1','swir_2']
    measurements_MAD=['smad','emad','bcmad']

    #connect to the datacube
    dc = datacube.Datacube(app='feature_layers')

    # load data
    ds = load_ard(dc=dc,
                  products=['s2_l2a'],
                  measurements=measurements,
                  group_by='solar_day',
                  verbose=False,
                  #mask_filters=[("opening", 2)], # morphological opening by 2 pixels to remove small masked regions
                  **query)

    # calcualte NDVI
#     ds_index = calculate_indices(ds,index=['NDVI'],drop=False,collection='s2')
    ds_index = calculate_indices(ds,index=['NDVI'],drop=False,satellite_mission='s2')
    # interpolate nodata using mean of previous and next observation
#     ds=ds.interpolate_na(dim='time',method='linear',use_coordinate=False)

    # calculate geomedians within each two-month interval
    ds_geomedian=ds_index.resample(time='2MS').map(xr_geomedian)

    # stack multi-temporal measurements and rename them
    n_time=ds_geomedian.dims['time']
    list_measurements=list(ds_geomedian.keys())
    list_stack_measures=[]
#     ds_stacked=None
    for j in range(len(list_measurements)):
        for k in range(n_time):
            variable_name=list_measurements[j]+'_'+str(k)
            measure_single=ds_geomedian[list_measurements[j]].isel(time=k).rename(variable_name)
            list_stack_measures.append(measure_single)
    ds_stacked=xr.merge(list_stack_measures,compat='override')

    # load semiannual MADs
    ds_mads=dc.load(product='gm_s2_semiannual',
                    measurements=measurements_MAD,
                    **query
                   )
    # stack multi-temporal measurements and rename them
    n_time=ds_mads.dims['time']
    list_measurements=list(ds_mads.keys())
    for j in range(len(list_measurements)):
        for k in range(n_time):
            variable_name=list_measurements[j]+'_'+str(k)
            measure_single=ds_mads[list_measurements[j]].isel(time=k).rename(variable_name)
            ds_stacked=xr.merge([ds_stacked,measure_single],compat='override')
    return ds_stacked

### set up dask cluster for parallel processing

In [ ]:
# Set up a dask cluster
create_local_dask_cluster(n_workers=3)

### run prediction for all tiles and export geotiffs

In [ ]:
# iterate through each tile
for i in range(len(tile_bboxes)):
    minx,miny,maxx,maxy=tile_bboxes.iloc[i]
    print('bounding box for tile ',i,': minx: ',minx,'miny: ',miny,'maxx: ',maxx,'maxy: ',maxy)

    # load Sentinel-2 data
    query = {
        'x': (minx,maxx),
        'y': (miny,maxy),
        'time': ('2021-01', '2021-12'),
#         'measurements': measurements,
        'resolution': (-10, 10),
        'crs':'epsg:4326',
        'output_crs':crs,
#         'dask_chunks' : {'x':-1, 'y':-1}
        'dask_chunks' : {'x':2000, 'y':2000} # change this based on your tile size and sandbox instance
    }

    # calculate features
    all_data = feature_layers(query) # making sure feature order is the same to training data
    print('stacked Sentinel-2 dataset:\n',all_data)

    # timing how long it takes for the prediction
    start_time = time.time()
    predicted = predict_xr(rf_models,all_data,persist=False,clean=True).compute() # predict classes of all data using the RF model
    print("%s seconds spent on predicting" % (time.time() - start_time))

    # write final prediction as cog file
    print('writing cog file...')
    outname_prediction='Results/Land_cover_prediction_15pct_td_Mozambique_50km_sample_region_'+str(i)+'.tif'
    write_cog(predicted.Predictions, outname_prediction, overwrite=True)

### merge all tiles

In [1]:
! gdal_merge.py -o Results/Land_cover_prediction_15pct_td_Mozambique_50km_sample_regions_mosaic.tif -co COMPRESS=Deflate -ot Byte Results/Land_cover_prediction_15pct_td_Mozambique_50km_sample_region_*.tif

0...10...20...30...40...50...60...70...80...90...100 - done.
